# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **model analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `modelproject.py` file includes a function which could be used multiple times in this notebook.

Imports and set magics:

In [2]:
import numpy as np
from scipy import optimize
import sympy as sm
from types import SimpleNamespace


import pandas as pd 
import matplotlib.pyplot as plt

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

# Principal Agent Model


Consider an economy that consists of a principal that wish to hire workers. There are two types of workers, resepctivly high and low productivity workers. The principal faces the following maximization problem: 
$$
\begin{align*}
\max_{w_H,e_H,w_L,e_l} \pi  & = q(py_H+\alpha e_H -w_h)+(1-q)(py_L+\alpha e_L-w_L)\\
& \text{s.t.} \\
\tag{$IR_H$}u_H(w_h,e_H) & \ge r_H \\ 
\tag{$IR_L$}u_L(w_L,e_L) & \ge r_L \\ 
\tag{$IC_H$}u_H(w_H,e_H) & \ge u(w_L,e_L)\\
\tag{$IC_L$}u_L(w_L,e_L) & \ge u_L(w_H,e_H)
\end{align*}
$$
where $w$ is the wage rate, $e$ is the level of education for the workers, $p$ is the price and $y$ is the production. $r$ is the outside utility. The subscript of either L or H denotes if the agent has high or low productivity.


The individual rationality constraints ($IR_H$/$IR_L$) ensures that the agents accept the offerd contract. This implies that the utility of accepting the contract is higher than the outside utility. The incentive compatible constraints ($IC_H$/$IC_L$) ensures that the workers only accept the contract that is meant for them, i.e low productivity workers don't accept the contract meant for high productivity workers (and in principle also vise versa). 

We have assumed the functional forms of the various functions given below:

$$
\begin{align*}
u_H & = w - b_H * e_H \\
u_L & = w - b_L * e_L \\
R_H & = y_H + \alpha * e_H \\
R_L & = y_L + \alpha * e_L \\
\end{align*}
$$
, where $R_H$ and $R_L$ denotes the revenue created for the firm by high productivity workers and lwo productivity workers, respectively.

Note also that we have normalized $p=1$.

In [4]:
#defining the baseline parameters used in the model and creating a space for solutions
def __init__(self):

    par = self.par = SimpleNamespace()
    sol = self.sol =SimpleNamespace()
    
    par.alpha = 0.5
    par.q = 0.5
    par.bl = 3
    par.bh = 1
    par.yl = 100
    par.yh = 200




## Analytical solution

If your model allows for an analytical solution, you should provide here.

You may use Sympy for this. Then you can characterize the solution as a function of a parameter of the model.

To characterize the solution, first derive a steady state equation as a function of a parameter using Sympy.solve and then turn it into a python function by Sympy.lambdify. See the lecture notes for details. 

## Numerical solution

You can always solve a model numerically. 

Define first the set of parameters you need. 

Then choose one of the optimization algorithms that we have gone through in the lectures based on what you think is most fitting for your model.

Are there any problems with convergence? Does the model converge for all starting values? Make a lot of testing to figure these things out. 

# Further analysis

Make detailed vizualizations of how your model changes with parameter values. 

Try to make an extension of the model. 

# Conclusion

Add concise conclusion. 